In [1]:
import shutil

shutil.rmtree('templates', ignore_errors=True)
shutil.rmtree('static', ignore_errors=True)

In [2]:
# Save the Flask app code
with open('app.py', 'w') as f:
    f.write("""
from flask import Flask, request, render_template, jsonify
import pandas as pd
import numpy as np

app = Flask(__name__)

# Load the saved similarity matrix and dataset
combined_similarity_matrix = np.load('/content/combined_similarity_matrix.npy')
df = pd.read_csv('/content/processed_dataset.csv')

def recommend_songs(song_title, similarity_matrix, df, n_recommendations=5, similarity_threshold=0.5):
    try:
        song_index = df[df['Title'] == song_title].index[0]  # Ensure 'Title' is the correct column name
        similarity_scores = list(enumerate(similarity_matrix[song_index]))
        similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
        recommended_indices = [i[0] for i in similarity_scores[1:] if i[1] >= similarity_threshold][:n_recommendations]
        return df['Title'].iloc[recommended_indices]
    except IndexError:
        return "Song title not found in the dataset."

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/recommend', methods=['GET'])
def recommend():
    song_title = request.args.get('song_title')
    recommendations = recommend_songs(song_title, combined_similarity_matrix, df)
    return render_template('recommendations.html', song_title=song_title, recommendations=recommendations)

if __name__ == '__main__':
    app.run()
    """)

# Create templates directory and save HTML files
!mkdir templates
with open('templates/index.html', 'w') as f:
    f.write("""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Music Recommendation System</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
</head>
<body>
    <div class="container">
        <h1>Welcome to the Music Recommendation System</h1>
        <form action="/recommend" method="get">
            <input type="text" name="song_title" placeholder="Enter song title">
            <button type="submit">Get Recommendations</button>
        </form>
    </div>
</body>
</html>
    """)

with open('templates/recommendations.html', 'w') as f:
    f.write("""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Recommendations for {{ song_title }}</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
</head>
<body>
    <div class="container">
        <h1>Recommendations for "{{ song_title }}"</h1>
        <ul>
            {% for song in recommendations %}
                <li>{{ song }}</li>
            {% endfor %}
        </ul>
        <a href="/">Go back</a>
    </div>
</body>
</html>
    """)

# Create static directory and save CSS file
!mkdir static
with open('static/styles.css', 'w') as f:
    f.write("""
body {
    font-family: 'Arial', sans-serif;
    background-color: #121212;
    color: #FFFFFF;
    margin: 0;
    padding: 0;
}

.container {
    max-width: 600px;
    margin: 50px auto;
    padding: 20px;
    background-color: #1DB954;
    border-radius: 8px;
    text-align: center;
}

h1 {
    font-size: 2em;
    margin-bottom: 20px;
}

form {
    margin-bottom: 20px;
}

input[type="text"] {
    padding: 10px;
    width: 80%;
    border: none;
    border-radius: 4px;
    margin-right: 10px;
}

button {
    padding: 10px 20px;
    border: none;
    background-color: #FFFFFF;
    color: #1DB954;
    border-radius: 4px;
    cursor: pointer;
}

button:hover {
    background-color: #1ED760;
}

ul {
    list-style-type: none;
    padding: 0;
}

li {
    background-color: #FFFFFF;
    color: #1DB954;
    padding: 10px;
    margin: 5px 0;
    border-radius: 4px;
}

a {
    color: #FFFFFF;
    text-decoration: none;
}

a:hover {
    text-decoration: underline;
}
    """)

# Set up Ngrok and run the Flask app
from pyngrok import ngrok

# Set up the Ngrok authtoken
!ngrok authtoken ---

# Terminate open tunnels if exist
ngrok.kill()

# Expose the app on port 5000
public_url = ngrok.connect(5000)
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:5000/\"".format(public_url))

# Run the Flask app
!FLASK_APP=app.py flask run --host=0.0.0.0 --port=5000

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 * ngrok tunnel "NgrokTunnel: "https://83fd-35-196-58-115.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000/"
 * Serving Flask app 'app.py'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Jun/2024 16:17:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2024 16:17:09] "GET /static/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2024 16:17:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Jun/2024 16:18:04] "GET /recommend?song_title=Empire+State+Of+Mind HTTP/1.1" 200 -
127.0.0.1 - - [11/Jun/2024 16:18:05] "GET /static/styles.css HTTP/1.1" 304 -
Exception ignored in: <module 'threading' from '/usr/lib/python3.10/threading.py'>
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1518, in _shutdown
    def _shutdown():
KeyboardInterrupt: 
